# DTale


After completing a train run in nnUNet, the validation metrics are stored in a JSON file named `summary.json` in the validation folder. We can load the file and visualize the metrics with [DTale](https://github.com/man-group/dtale) using the following code:

In [ ]:
import json
import pandas as pd
from pathlib import Path

task_name = "Task09_Spleen"
task_id = "009"
trainer_name = "nnUNetTrainer"
plans_identifier = "nnUNetPlans"
fold = 0
configuration = "3d_fullres"
summary_file = f"/home/maia-user/shared/nnUNet/nnUNet_trained_models/Dataset{task_id}_{task_name}/nnUNetTrainer__{plans_identifier}__{configuration}/fold_{fold}/validation/summary.json"

Or, to use the MLflow artifact store to fetch the file directly from the MLflow server:

In [ ]:
import mlflow
import os

mlflow.set_tracking_uri("https://aida-workshop.maia-small.cloud.cbh.kth.se/mlflow")
# Artifact URI from your example
artifact_uri = ""

# Local path to store the downloaded file
local_path = mlflow.artifacts.download_artifacts(artifact_uri)

print(f"Artifact downloaded to: {local_path}")

In [ ]:
summary_file = local_path

In [ ]:
with open(summary_file) as f:
    summary = json.load(f)

In [ ]:
config_dict = {
    "label_dict":{
        "Background": 0,
        "Spleen": 1
    },
    "label_suffix": ".nii.gz"
}

In [ ]:
df = []

label_to_name = {v: k for k, v in config_dict["label_dict"].items()}

for case in summary['metric_per_case']:
    for label_id in case['metrics']:
        for metric in case['metrics'][label_id]:
           
            df.append({
                "Case": Path(case['reference_file']).name[:-len(config_dict["label_suffix"])],
                "Label": label_to_name[int(label_id)],
                "Metric": metric,
                "Value": case['metrics'][label_id][metric]
            })

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df.groupby(["Metric"]).describe()

In [ ]:
import dtale
import dtale.app as dtale_app

In [ ]:
dtale_app.JUPYTER_SERVER_PROXY = False
d = dtale.show(df,host="0.0.0.0",port=40000)

In [ ]:
from IPython.display import Markdown
from IPython.core.magic import register_cell_magic



DTALE_URL = d._main_url
@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))

In [ ]:
%%markdown

[DTale]({DTALE_URL})